In [1]:
"""
The file contains the code used to preprare the datasets for the analysis files.
"""

'\nThe file contains the code used to preprare the datasets for the analysis files.\n'

In [2]:
# Library imports
import pandas as pd; import numpy as np;

In [3]:
# Read in datasets
listings_full_df = pd.read_csv("../Data/listings.csv", low_memory=False)

In [4]:
# Which columns have high percentages of missing data
cols_null_50 = listings_full_df.columns[listings_full_df.isnull().mean() > .5].tolist()
cols_null_75 = listings_full_df.columns[listings_full_df.isnull().mean() > .75].tolist()
cols_null_100 = listings_full_df.columns[listings_full_df.isnull().mean() == 1.].tolist()

In [5]:
# Remove all vars that have 75% missing or above
listings_full_df = listings_full_df.drop(cols_null_75, axis=1)

In [6]:
# Which columns have no missing values
cols_no_nulls = listings_full_df.columns[listings_full_df.isnull().mean() == 0].tolist()

In [7]:
# Which columns have low cardinality
cols_low_cardinality = listings_full_df.columns[listings_full_df.nunique() == 1].tolist()

In [8]:
# Remove columns with low cardinality
listings_full_df = listings_full_df.drop(cols_low_cardinality, axis=1)

In [9]:
# Get columns needed from listings data and sort neighbourhoods into side of city
listings_1_df = listings_full_df[['id', 'neighbourhood', 'neighbourhood_cleansed', 'smart_location', 'property_type', 'room_type']].copy()

european_n = ['Fatih', 'Zeytinburnu', 'Bakirkoy', 'Kucukcekmece', 'Avcilar', 'Bahcelievler', 'Arnavutkoy', 'Bagcilar', 'Basaksehir', 'Bayrampasar', 'Besiktas',
             'Beylikduzu', 'Beyoglu', 'Buyukcekmece', 'Catalca', 'Esenler', 'Esenyurt', 'Eyup', 'Gaziosmanpasa', 'Gungoren', 'Kagithane', 'Sariyer', 'Silivri', 'Sisli',
             'Sultangazi']
asian_n = ['Atasehir', 'Beykoz', 'Cekmekoy', 'Kadikoy', 'Kartal', 'Maltepe', 'Pendik', 'Sancaktepe', 'Sile', 'Sultanbeyli', 'Tuzla', 'Umraniye', 'Uskudar']
listings_1_df['Side'] = listings_1_df.neighbourhood_cleansed.apply(lambda x: 'European' if x in european_n else 'Asian')

In [10]:
# Read in dataset
calendar_df = pd.read_csv("../Data/calendar.csv", low_memory=False)

In [11]:
# Filter out listings that have a min nights stay greater than 5
calendar_fil_df = calendar_df.query("minimum_nights <= 5").copy()

In [12]:
# Convert date feature to date type and create new columns
calendar_fil_df.date = pd.to_datetime(calendar_fil_df.date)
calendar_fil_df['Month'] = calendar_fil_df.date.dt.month
calendar_fil_df['MonthString'] = calendar_fil_df.Month.map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct',
                                                           11: 'Nov', 12: 'Dec'})
calendar_fil_df['Year'] = calendar_fil_df.date.dt.year

In [13]:
# Filter ids that have missing price data and investigate
id_w_missing_prices = calendar_fil_df.loc[calendar_fil_df.isnull().any(axis=1), 'listing_id'].unique().tolist()

In [14]:
# Remove rows with missing price data
calendar_fil_df = calendar_fil_df.dropna(subset=['price', 'adjusted_price'], how='any', axis=0)

In [15]:
# Create new column that converts price to float type
calendar_fil_df['adjusted_price_num'] = calendar_fil_df.adjusted_price.str.replace('[$,]', '').astype(float)

In [16]:
# Create pivot table showing price for each month for each listing
calendar_price_df = calendar_fil_df.pivot_table(index='listing_id', columns='MonthString', values='adjusted_price_num', aggfunc='mean').reset_index()
calendar_price_df = calendar_price_df[['listing_id', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

In [17]:
# merge prepared listing and prepared calendar data ready for analysis of prices
final_df_1 = listings_1_df.merge(calendar_price_df, how='inner', left_on='id', right_on='listing_id').drop(['listing_id'], axis=1)

In [21]:
for i in final_df_1.select_dtypes(include='int').columns:
    final_df_1[i] = final_df_1[i].astype('int16')
    
for i in final_df_1.select_dtypes(include='float').columns:
    final_df_1[i] = final_df_1[i].astype('float16')

In [24]:
# save dataset to csv
final_df_1.to_csv('../Data/Price_Data.csv', index=False)

In [25]:
# filter listings data and include certain columns
listings_2_df = listings_full_df[['id', 'summary', 'description', 'neighbourhood_cleansed', 'review_scores_rating']]

In [26]:
# read in dataset
reviews_df = pd.read_csv("../Data/reviews.csv", low_memory=False)

In [27]:
# merge prepared listings data with reviews data ready for analysis of reviews file
final_df_2 = reviews_df.merge(listings_2_df, how='inner', left_on='listing_id', right_on='id').drop(['id_x', 'id_y'], axis=1)

In [29]:
final_df_2.drop(['summary', 'description', 'review_scores_rating', 'reviewer_id', 'reviewer_name'], axis=1, inplace=True)

In [30]:
# save the daaset to csv
final_df_2.to_csv("../Data/Review_Data.csv", index=False)